In [24]:
import requests
import pandas as pd

from datetime import datetime, timedelta
import time
import json
import re

In [3]:
today = datetime.now()
query = {
    "startDate": (today - timedelta(days=90)).strftime("%Y%m%d"),
    "endDate": today.strftime("%Y%m%d"),
    "code": "",
    "cate": "",
    "type": "all",
    "order": "date",
    "id": "",
    "response": "json",
}

res = requests.get("https://www.tpex.org.tw/www/zh-tw/bulletin/attention", params=query)
if res.status_code != 200:
    print("request failed")
else:
    print("request successfully")

request successfully


In [53]:
datas=[]
with open("TSE_notice.json", "r", encoding="utf-8") as f:
    TSE_data = json.load(f)
    datas.append(TSE_data)
with open("OTC_notice.json", "r", encoding="utf-8") as f:
    OTC_data = json.load(f)
    datas.append(OTC_data)
for data in datas:
    if data==TSE_data:
        data = pd.DataFrame(data["data"], columns=data["fields"])
        print("TSE_data processed")
    elif data==OTC_data:
        data = pd.DataFrame(data['tables'][0]["data"], columns=data['tables'][0]["fields"])
        print("OTC_data processed")

TSE_data processed
OTC_data processed


In [ ]:

data = pd.DataFrame(data['tables'][0]["data"], columns=data['tables'][0]["fields"])
data["證券代號"] = data["證券代號"].astype(str)
data = data[data["證券代號"].str.len() == 4]
data = data[data["證券代號"] > "1000"]
refs = {
    "第一款": 1,
    "第二款": 2,
    "第三款": 3,
    "第四款": 4,
    "第五款": 5,
    "第六款": 6,
    "第七款": 7,
    "第八款": 8,
    "第九款": 9,
    "第十款": 10,
    "第十一款": 11,
    "第十二款": 12,
    "第十三款": 13
}

data["注意交易資訊"] = data["注意交易資訊"].apply(
    lambda x: [
        refs[m]
        for m in re.findall(r'[（(](第[一二三四五六七八九十百零]+款)[)）]', str(x))
        if m in refs
    ]
)
data = data.drop(columns=['link'])

def convert_date(date_str):
    year = int(date_str[:3]) + 1911
    month = int(date_str[4:6])
    day = int(date_str[7:9])
    return f"{year}-{month:02d}-{day:02d}"
data["公告日期"] = data["公告日期"].apply(convert_date)
# data["注意交易資訊"] = data["注意交易資訊"].apply(lambda x: json.dumps(x, ensure_ascii=False))
data

,編號,證券代號,證券名稱,累計,注意交易資訊,公告日期,收盤價,本益比
0,1,3162,精確,22,"[3, 4]",114/08/29,67.70,27.63
1,1,3163,波若威,22,[13],114/08/29,248.00,94.30
2,1,3219,倚強科,22,"[1, 2, 2, 4]",114/08/29,108.00,128.57
3,1,3297,杭特,22,[10],114/08/29,66.10,472.14
5,1,3685,元創精密,22,"[1, 3]",114/08/29,20.75,N/A
6,1,4541,晟田,22,"[1, 3, 4, 13]",114/08/29,60.60,47.34
7,1,4760,勤凱,22,"[1, 4]",114/08/29,193.00,34.46
8,1,4772,台特化,22,[6],114/08/29,292.00,96.05
9,1,4931,新盛力,22,[6],114/08/29,168.50,68.78
10,1,4991,環宇-KY,22,[6],114/08/29,160.50,458.57


,編號,證券代號,證券名稱,累計,注意交易資訊,公告日期,收盤價,本益比
0,1,3162,精確,22,"[3, 4]",2025-08-29,67.70,27.63
1,1,3163,波若威,22,[13],2025-08-29,248.00,94.30
2,1,3219,倚強科,22,"[1, 2, 2, 4]",2025-08-29,108.00,128.57
3,1,3297,杭特,22,[10],2025-08-29,66.10,472.14
5,1,3685,元創精密,22,"[1, 3]",2025-08-29,20.75,N/A
6,1,4541,晟田,22,"[1, 3, 4, 13]",2025-08-29,60.60,47.34
7,1,4760,勤凱,22,"[1, 4]",2025-08-29,193.00,34.46
8,1,4772,台特化,22,[6],2025-08-29,292.00,96.05
9,1,4931,新盛力,22,[6],2025-08-29,168.50,68.78
10,1,4991,環宇-KY,22,[6],2025-08-29,160.50,458.57
